<a href="https://colab.research.google.com/github/vvud/notebooks/blob/main/llama_2_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary dependencies
!pip install transformers
!pip install ctransformers
!pip install streamlit
!pip install tensorflow
!pip install torch
!pip install bitsandbytes
!pip install accelerate

In [ ]:
!pip install huggingface_hub
from huggingface_hub import interpreter_login
interpreter_login()

### Create file

In [30]:
%%writefile app.py
import streamlit as st
import streamlit as st
import os
from ctransformers import AutoModelForCausalLM
# from transformers import AutoTokenizer, AutoModelForCausalLM

# App title
st.set_page_config(page_title="🦙💬 Llama 2 Chatbot")

PROMPT = "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'."
# You are an AI Coding assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'AI Coding assistant'.

@st.cache_resource()
def ChatModel(temperature, top_p):
    return AutoModelForCausalLM.from_pretrained(
        'TheBloke/Llama-2-7B-Chat-GGML',
        model_type='llama',
        temperature=temperature,
        top_p = top_p)

#def ChatModel(temperature, top_p):
#    return AutoModelForCausalLM.from_pretrained(
#        'meta-llama/Llama-2-7b-chat-hf',
#        model_type='llama',
#        temperature=temperature,
#        top_p = top_p,
#        device_map='auto', load_in_8bit=True)

# Custom st sidebar
with st.sidebar:
    st.title('🦙💬 Llama 2 Chatbot')
    st.subheader('Models and parameters')

    if 'prompt' not in st.session_state:
        st.session_state['prompt'] = PROMPT

    #Model hyper parameters:
    temperature = st.sidebar.slider('temperature', min_value=0.01, max_value=2.0, value=0.1, step=0.01)
    top_p = st.sidebar.slider('top_p', min_value=0.01, max_value=1.0, value=0.9, step=0.01)
    # max_length = st.sidebar.slider('max_length', min_value=64, max_value=4096, value=2048, step=8)

    PROMPT = st.sidebar.text_area('Prompt:', PROMPT, height=150)

    chat_model = ChatModel(temperature, top_p)
    # st.markdown('📖 Learn how to build this app in this [blog](#link-to-blog)!')

# Store LLM generated responses
if "messages" not in st.session_state.keys():
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]

# Display or clear chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

def clear_chat_history():
    # st.session_state['chat_dialogue'] = []
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]
st.sidebar.button('Clear Chat History', on_click=clear_chat_history)

# Function for generating LLaMA2 response
def generate_llama2_response(prompt_input):
    string_dialogue = PROMPT
    for dict_message in st.session_state.messages:
        if dict_message["role"] == "user":
            string_dialogue += "User: " + dict_message["content"] + "\n\n"
        else:
            string_dialogue += "Assistant: " + dict_message["content"] + "\n\n"
    print (string_dialogue)
    output = chat_model(f"prompt {string_dialogue} {prompt_input} Assistant: ")
    return output

# User-provided prompt
if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

# Generate a new response if last message is not from assistant
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = generate_llama2_response(prompt)
            placeholder = st.empty()
            full_response = ''
            for item in response:
                full_response += item
                placeholder.markdown(full_response)
            placeholder.markdown(full_response)
    message = {"role": "assistant", "content": full_response}
    st.session_state.messages.append(message)


Writing app.py


### Install localtunnel and run

In [31]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.42s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [32]:
!streamlit run /content/app.py &> /content/logs.txt &

In [33]:
path = '/content/logs.txt'
with open(path, 'r') as f:
  print(f.read())




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.173.73.245:8501




In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.28s
your url is: https://cool-birds-hug.loca.lt
